# Aplicando um modelo de random forest para tarefas de classificação

No exemplo a baixo, validamos a utilização de um random forest aplicado ao exemplo de sobreviventes no Titanic

In [84]:
import json
import csv
from validation.data import Modeling

In [85]:
from validation.data import Files

In [49]:
from joblib import dump, load

In [50]:
fileManipulation = Files("make_json",["Datassets/tested.csv","Datassets/titanic.json","PassengerId"])

In [51]:
fileManipulation.jsonFile

{'892': {'PassengerId': '892',
  'Survived': '0',
  'Pclass': '3',
  'Name': 'Kelly, Mr. James',
  'Sex': 'male',
  'Age': '34.5',
  'SibSp': '0',
  'Parch': '0',
  'Ticket': '330911',
  'Fare': '7.8292',
  'Cabin': '',
  'Embarked': 'Q'},
 '893': {'PassengerId': '893',
  'Survived': '1',
  'Pclass': '3',
  'Name': 'Wilkes, Mrs. James (Ellen Needs)',
  'Sex': 'female',
  'Age': '47',
  'SibSp': '1',
  'Parch': '0',
  'Ticket': '363272',
  'Fare': '7',
  'Cabin': '',
  'Embarked': 'S'},
 '894': {'PassengerId': '894',
  'Survived': '0',
  'Pclass': '2',
  'Name': 'Myles, Mr. Thomas Francis',
  'Sex': 'male',
  'Age': '62',
  'SibSp': '0',
  'Parch': '0',
  'Ticket': '240276',
  'Fare': '9.6875',
  'Cabin': '',
  'Embarked': 'Q'},
 '895': {'PassengerId': '895',
  'Survived': '0',
  'Pclass': '3',
  'Name': 'Wirz, Mr. Albert',
  'Sex': 'male',
  'Age': '27',
  'SibSp': '0',
  'Parch': '0',
  'Ticket': '315154',
  'Fare': '8.6625',
  'Cabin': '',
  'Embarked': 'S'},
 '896': {'PassengerId': 

In [52]:
datasset = fileManipulation.jsonFile

In [53]:
datasset = [datasset[i] for i in datasset]

In [54]:
datasset[0].keys()

dict_keys(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])

In [55]:
titatic_data = Modeling(datasset,['Survived'],drops=["Cabin","Name","Embarked","Ticket","PassengerId"]) # Retirando colunas que não servem

In [56]:
titatic_data.X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
258,1,0,18,0,0,12
31,1,1,25,2,0,82
405,1,1,20,0,0,17
236,0,1,74,1,0,150
162,1,0,27,0,0,14
...,...,...,...,...,...,...
38,2,1,26,0,0,166
170,2,1,0,0,0,127
37,2,0,21,0,0,157
79,2,0,25,0,0,135


# Aplicando o modelo:

In [57]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report

In [58]:

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100, 
    random_state=42,
    max_depth=10,  # Limitar a profundidade das árvores
    min_samples_split=10,  # Mínimo de amostras para dividir um nó
    min_samples_leaf=5  # Mínimo de amostras em um nó folha
)
 
# Train the classifier
rf_classifier.fit(titatic_data.X_train, titatic_data.y_train)


c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=10,
                       random_state=42)

In [59]:
# Make predictions on the test set
y_pred = rf_classifier.predict(titatic_data.X_test)

In [60]:
titatic_data.y_test

,Survived
384,0
0,0
109,0
334,0
151,0
...,...
223,0
253,0
409,1
44,1


In [61]:
y_pred

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0',
       '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1', '0', '1',
       '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1',
       '1', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1',
       '1', '0', '0', '1', '1', '0'], dtype=object)

In [62]:
# Avaliar desempenho no conjunto de treino
train_predictions = rf_classifier.predict(titatic_data.X_train)

train_accuracy = accuracy_score(titatic_data.y_train, train_predictions)
print(f"Train Accuracy: {train_accuracy:.2f}")

# Avaliar desempenho no conjunto de teste
test_accuracy = accuracy_score(titatic_data.y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Comparar as métricas
if train_accuracy > test_accuracy + 0.05:
    print("O modelo pode estar sofrendo de overfitting.")
else:
    print("O modelo não parece estar sofrendo de overfitting.")


Train Accuracy: 1.00
Test Accuracy: 1.00
O modelo não parece estar sofrendo de overfitting.


In [63]:
from sklearn.model_selection import cross_val_score

# Realizar validação cruzada com 5 folds
cv_scores = cross_val_score(rf_classifier, titatic_data.encoded_features, titatic_data.target, cv=5)

print("Cross-Validation Scores:", cv_scores)
print(f"Mean CV Score: {cv_scores.mean():.2f}")
print(f"Std CV Score: {cv_scores.std():.2f}")


c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwa

Cross-Validation Scores: [1. 1. 1. 1. 1.]
Mean CV Score: 1.00
Std CV Score: 0.00


In [64]:
titatic_data.X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
258,1,0,18,0,0,12
31,1,1,25,2,0,82
405,1,1,20,0,0,17
236,0,1,74,1,0,150
162,1,0,27,0,0,14
...,...,...,...,...,...,...
38,2,1,26,0,0,166
170,2,1,0,0,0,127
37,2,0,21,0,0,157
79,2,0,25,0,0,135


In [65]:
print(titatic_data.dataframe["Sex"][6])

female


In [66]:
titatic_data.X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
258,1,0,18,0,0,12
31,1,1,25,2,0,82
405,1,1,20,0,0,17
236,0,1,74,1,0,150
162,1,0,27,0,0,14
...,...,...,...,...,...,...
38,2,1,26,0,0,166
170,2,1,0,0,0,127
37,2,0,21,0,0,157
79,2,0,25,0,0,135


In [67]:

# Para salvar o modelo
dump(rf_classifier, 'random_forest_model.joblib')

# Para carregar o modelo
rf_classifier = load('random_forest_model.joblib')

In [68]:
import pandas as pd

PassengerId	Pclass	Sex	Age	SibSp	Parch	Ticket	Fare

	186	      2	 	1	 42	  0	       0	173		166

In [69]:
# Assuming 'data' is your dictionary containing the data
data = {
    "Pclass": [2],
    "Sex": [1],
    "Age": [42],
    "SibSp": [0],
    "Parch": [0],
    "Fare": [166]
}



In [70]:
# Define index and columns for DataFrame
index = [0]  # Assuming a single row DataFrame
columns = list(data.keys())

# Create DataFrame
df = pd.DataFrame(data, index=index, columns=columns)

In [71]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare
0,2,1,42,0,0,166


In [72]:
teste_new_pred = rf_classifier.predict(df)

In [73]:
teste_new_pred

array(['0'], dtype=object)

In [74]:
X_new = pd.DataFrame({
    'Pclass': [3, 1],
    'Sex': [1, 0], #1=male 0=female
    'Age': [100, 22],
    'SibSp': [6, 1],
    'Parch': [1, 0],
    'Fare': [15, 70]
})

y_new = [0, 1]


In [75]:
X_new

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,100,6,1,15
1,1,0,22,1,0,70


In [76]:
new_predictions = rf_classifier.predict(X_new)

In [77]:
new_predictions

array(['0', '1'], dtype=object)

In [78]:
new_predictions = new_predictions.astype(int)  # Converter previsões para inteiros


new_accuracy = accuracy_score(y_new, new_predictions)

print(f"Accuracy on new data: {new_accuracy:.2f}")

Accuracy on new data: 1.00


In [79]:
# Bagging
bagging_rf = BaggingClassifier(RandomForestClassifier(), n_estimators=50, max_samples=0.8, max_features=0.8)
bagging_rf.fit(titatic_data.X_train, titatic_data.y_train)
bagging_predictions = bagging_rf.predict(titatic_data.X_test)
bagging_accuracy = accuracy_score(titatic_data.y_test, bagging_predictions)
print("Bagging Accuracy:", bagging_accuracy)

# Random Subspace
random_subspace_rf = RandomForestClassifier(max_features='sqrt')  # Use 'sqrt' para a raiz quadrada do número total de features
random_subspace_rf.fit(titatic_data.X_train, titatic_data.y_train)
random_subspace_predictions = random_subspace_rf.predict(titatic_data.X_test)
random_subspace_accuracy = accuracy_score(titatic_data.y_test, random_subspace_predictions)
print("Random Subspace Accuracy:", random_subspace_accuracy)

c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\ensemble\_bagging.py:888: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Bagging Accuracy: 1.0


c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Subspace Accuracy: 1.0


In [80]:
random_subspace_rf_new_preds =  bagging_rf.predict(X_new)

In [81]:
random_subspace_rf_new_preds

array(['0', '1'], dtype=object)

In [82]:
bagging_predictions

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0',
       '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1', '0', '1',
       '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1',
       '1', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1',
       '1', '0', '0', '1', '1', '0'], dtype=object)

In [83]:
random_subspace_predictions

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0',
       '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1', '0', '1',
       '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1',
       '1', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1',
       '1', '0', '0', '1', '1', '0'], dtype=object)